## Example

In this simple example, we load an image, pre-process it, and classify it with a pretrained EfficientNet.

In [1]:
import sys
sys.path.append('../../')

import json
from PIL import Image

import torch
from torchvision import transforms

from efficientnet_pytorch import EfficientNet


In [3]:
model_name = 'efficientnet-b0'
image_size = EfficientNet.get_image_size(model_name) # 224
image_size

224

In [24]:
# Open image
import numpy as np
import imageio

file_name = '/Users/yanqixu/Documents/1.0.MasterCDS/project-summer/breast_img/0_L-CC.png'
loaded_image = np.array(imageio.imread(file_name)).astype(np.float32)

loaded_image = np.expand_dims(np.expand_dims(loaded_image, 0), 0).copy()
tensor_batch = torch.Tensor(loaded_image)
tensor_batch.shape

torch.Size([1, 1, 2944, 1920])

In [ ]:
# convert python 2D array into 4D torch tensor in N,C,H,W format
loaded_image = np.expand_dims(np.expand_dims(loaded_image, 0), 0).copy()
tensor_batch = torch.Tensor(loaded_image).to(device)

In [5]:
# Preprocess image
tfms = transforms.Compose([transforms.Resize(image_size), transforms.CenterCrop(image_size), 
                           transforms.ToTensor(),
                           transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])
img = tfms(img).unsqueeze(0)

In [6]:
img_new = torch.cat([torch.cat([img[:,:1,:,:],torch.zeros(1,1,224,256-224)],3),torch.zeros(1,1,256-224,256)],2)
img_new.shape

torch.Size([1, 1, 256, 256])

In [7]:
# Load class names
labels_map = json.load(open('labels_map.txt'))
labels_map = [labels_map[str(i)] for i in range(1000)]

In [8]:
# Classify with EfficientNet
model = EfficientNet.from_pretrained(model_name,in_channels=1)
model.eval()
with torch.no_grad():
    logits = model(img_new)
preds = torch.topk(logits, k=5).indices.squeeze(0).tolist()

print('-----')
for idx in preds:
    label = labels_map[idx]
    prob = torch.softmax(logits, dim=1)[0, idx].item()
    print('{:<75} ({:.2f}%)'.format(label, prob*100))

Loaded pretrained weights for efficientnet-b0
-----
window screen                                                               (33.92%)
fire screen, fireguard                                                      (18.37%)
hamper                                                                      (1.62%)
envelope                                                                    (1.31%)
radiator                                                                    (0.88%)


In [9]:
new_model = EfficientNet.from_name(model_name,in_channels=1)

In [13]:
new_model.forward(img_new)

tensor([[ 0.0876, -0.2644,  0.0608, -0.2696, -0.0770, -0.0583,  0.1254, -0.1443,
         -0.0052, -0.0525,  0.0054,  0.0856, -0.1198,  0.1088,  0.0589, -0.1457,
          0.2038,  0.2289, -0.0964,  0.0555,  0.0360,  0.0716,  0.1458,  0.0355,
         -0.0204,  0.0312,  0.0588,  0.1574, -0.1796, -0.0317, -0.0457,  0.1852,
          0.1554,  0.1885,  0.2013,  0.0034, -0.0473, -0.0688,  0.2937, -0.2701,
          0.1115, -0.0128,  0.0881, -0.1517, -0.0399, -0.2056,  0.0475, -0.0960,
         -0.1882, -0.0936, -0.3352,  0.0535, -0.1223,  0.0257,  0.0571,  0.3082,
          0.1474,  0.1286,  0.1966,  0.0241, -0.0349, -0.0922,  0.1512, -0.1469,
          0.1174,  0.0733, -0.0855, -0.1265, -0.0680,  0.1131,  0.1897,  0.0257,
          0.2418,  0.0014, -0.2239,  0.0150,  0.0538, -0.0550,  0.0012, -0.3482,
          0.2123, -0.0201, -0.0203,  0.0093,  0.0381, -0.2533,  0.0892, -0.0274,
         -0.0031,  0.0647, -0.0435, -0.1245, -0.0439,  0.2936, -0.1324, -0.0490,
         -0.1770, -0.3134, -